In [9]:
import networkx as nx
import pandas as pd
from PIL import Image, ImageDraw
import io
import random
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv('Alliances.csv')
G = nx.from_pandas_edgelist(df,'Side_a', 'Side_b')

# Load the edgelist from the CSV file, skipping rows with missing values
def load_network_from_csv(filename):
    df = pd.read_csv(filename)
    df = df.dropna(subset=['Side_a', 'Side_b'])  # Skip rows with missing 'grupo' or 'aliado'
    G = nx.Graph()  # Create an empty graph
    for _, row in df.iterrows():
        source = row['Side_a']
        target = row['Side_b']
        # Encode node labels as UTF-8 to handle special characters
        source = source.encode('utf-8').decode('utf-8')
        target = target.encode('utf-8').decode('utf-8')
        G.add_edge(source, target)
    return G

# Perform a cascading failure simulation
def cascading_failure_simulation(G):
    failed_nodes = []
    simulations = []

    while G.number_of_edges() > 0:
        simulations.append(G.copy())  # Save a copy of the current network state
        
        # Calculate degree centrality for remaining nodes
        node_degrees = dict(G.degree())
        
        # Find the node with the highest degree centrality among remaining nodes
        max_degree_node = max(node_degrees, key=node_degrees.get)
        
        # Highlight the node to be removed in the next step
        highlighted_node = max_degree_node
        
        # Fail the node with the highest degree centrality
        failed_nodes.append(max_degree_node)
        
        # Remove the node and its incident edges from the network
        G.remove_node(max_degree_node)
    
    return simulations, failed_nodes

# Load the network from CSV
edgelist_file = 'Alliances.csv'
network = load_network_from_csv(edgelist_file)

# Run the cascading failure simulation
simulations, removed_nodes = cascading_failure_simulation(network)

# Create a GIF to visualize each iteration with consistent node positions
frames = []
pos = nx.spring_layout(simulation, k=.25, iterations=50, seed=42)  # Initial layout positions
for i, simulation in enumerate(simulations):
    plt.figure(figsize=(30, 20))
    
    nx.draw_networkx_nodes(simulation, pos=pos, edgecolors='black', node_size=1600, node_color='blue')
    nx.draw_networkx_labels(simulation, pos, font_size=14, font_color='black')
    nx.draw_networkx_edges(simulation, pos=pos, width=4, edge_color='green')
    plt.title(f'Step {i + 1}: Node Removal (Degree Centrality)')

    
# Highlight the node to be removed in this step
    nx.draw_networkx_nodes(simulation, pos, nodelist=[removed_nodes[i]], edgecolors='black', node_color='red', node_size=2600)
    
    # Save the current plot as an image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    frames.append(Image.open(buf))
    plt.close()

# Save the frames as a GIF
frames[0].save('cascading_failure_animation.gif', bbox_inches='tight', save_all=True, append_images=frames[1:], duration=1000, loop=0)

# Print the list of removed nodes
print("Removed Nodes:", removed_nodes)

Removed Nodes: ['Cártel de Sinaloa', 'Cártel Jalisco Nueva Generación', 'Cárteles Unidos', 'Cártel de Santa Rosa de Lima', 'Cártel Nueva Plaza', 'La Unión León', 'Los Zambada', 'Los Viagra', 'Gente Nueva', 'La Nueva Empresa', 'Los Salazar', 'Los Guzmán', 'Organización criminal de la familia Meza Flores', 'Los Negros', 'Los Escudero', 'La Línea', 'La Unión Tepito', 'Sindicato Libertad', 'La Nueva Familia Michoacana', 'Los Arzate', 'Familia Núñez Meza', 'Los Zetas Vieja Escuela', 'Cártel de Oaxaca', 'Cártel de Los Arellano Félix', 'Los Paredes', 'Los Páez', 'Los Rojos', 'Cártel de Tepalcatepec', 'Banda de El América', 'Banda del Gato', 'Cártel de Ensenada', 'Fuerzas Especiales Grupo Sombra', 'Cártel de Tláhuac', 'Cártel del Pelón de Playas', 'Los Rodolfos', 'Los Merino', 'Guerreros Unidos', 'Los Pulido', 'Los Salgueiro', 'Fuerza Rural Comunitaria de Pueblos del Norte de Culiacán', 'Fuerzas Especiales Mexicles', 'Los Cazadores', 'La Ronda 88', 'Los Artistas Asesinos', 'Los Peques', 'Los T